In [1]:
# Import app
import pickle
from collections import Counter

from celery import group
from celery import chain
import pandas as pd

from celery_basics import app
from celery_basics import add
from celery_basics import make_pi
from celery_basics import apply_function
from celery_basics import apply_counter

#### Sum 1 to 2

In [2]:
res = add.apply_async(args=(1, 2))
results = res.get()
results

3

#### Calculate pi in range 1 to 10**5

In [3]:
%%timeit
job = group(make_pi.subtask((10**x, )) for x in range(0,6))
job_result = job.delay()
results = job_result.get()
print(results)

[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
1 loop, best of 3: 5.43 s per loop


In [4]:
%%timeit
pis = []
for x in range(0,6):
    pis.append(make_pi(10**x))
print(pis)

[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
[4.0, 3.0418396189294032, 3.1315929035585537, 3.140592653839794, 3.1414926535900345, 3.1415826535897198, 3.1415916535897743, 3.1415925535897915]
1 loop, best of 3: 5.94 s per loop


#### Sum the numbers between the 1 to 1000 twice

In [5]:
%%timeit
jobs = chain(add.si(i, i) for i in range(1,1001))
job_result = jobs.apply_async()
results = job_result.get()
print(results)

2000
2000
2000
2000
1 loop, best of 3: 48.5 s per loop


In [6]:
%%timeit
results = sum([add(i, i) for i in range(1,1001)])
print(results)

1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000
1001000


#### Counter of the words in reviews summaries

In [8]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=5000, nrows=50000)
counter = Counter()

In [9]:
for review in reviews:
    counter += review.summary.apply(lambda x :Counter(str(x).split(" "))).values.sum()

In [10]:
counter.most_common()

[('the', 32363),
 ('game', 25316),
 ('a', 20637),
 ('of', 19285),
 ('Great', 17458),
 ('for', 16711),
 ('to', 14127),
 ('and', 13869),
 ('A', 13424),
 ('but', 13302),
 ('is', 11602),
 ('The', 9684),
 ('Game', 9606),
 ('it', 9544),
 ('I', 9450),
 ('great', 8077),
 ('this', 7846),
 ('good', 7361),
 ('Good', 7211),
 ('fun', 6701),
 ('not', 6523),
 ('Fun', 6105),
 ('in', 5983),
 ('Not', 5934),
 ('best', 5890),
 ('Best', 5678),
 ('with', 5563),
 ('you', 5429),
 ('on', 4989),
 ('as', 4792),
 ('This', 4185),
 ('', 3638),
 ('games', 3605),
 ('One', 3482),
 ('have', 3470),
 ('-', 3348),
 ('like', 3289),
 ('than', 3233),
 ('game,', 3085),
 ('my', 3078),
 ('Very', 3062),
 ('Awesome', 2856),
 ('better', 2825),
 ('be', 2803),
 ('that', 2746),
 ('one', 2704),
 ('all', 2652),
 ("It's", 2591),
 ('love', 2512),
 ('game!', 2501),
 ('It', 2354),
 ('game.', 2332),
 ('your', 2295),
 ('ever', 2275),
 ('2', 2232),
 ('what', 2212),
 ('an', 2199),
 ('Excellent', 2173),
 ('so', 2164),
 ('very', 2144),
 ('at', 2

#### Counter of the words in reviews summaries with Celery

In [11]:
reviews = pd.read_csv("reviews.csv", encoding="utf-8", iterator=True, chunksize=5000, nrows=50000)

In [12]:
jobs =[]
for review in reviews:
    job = apply_counter.apply_async(args=(pickle.dumps(review),))
    jobs.append(job)
counter = Counter()
for job in jobs:
    counter += pickle.loads(job.get()) 

In [13]:
counter.sum().most_common()

AttributeError: 'Counter' object has no attribute 'sum'